In [1]:
# Install required libraries
!pip install datasets transformers seqeval scipy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=133ec76d4fc417b1de0d678b2c6ed69a8540d13d9b5d341d39d9511d59ecebbf
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency reso

In [2]:

import json
import pandas as pd
import torch
from scipy.stats import spearmanr
from tqdm import tqdm
import numpy as np
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForTokenClassification, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [3]:
# Upload un labled train data, validation data and sample label files
from google.colab import files
uploaded = files.upload()  # Upload your dataset files


Saving mushroom.en-train_nolabel.v1.jsonl to mushroom.en-train_nolabel.v1.jsonl
Saving mushroom.en-val.v1.jsonl to mushroom.en-val.v1.jsonl
Saving sample.txt to sample.txt


In [4]:
"""
# Initialize lists for train and test data
train_data_all = []
test_data_all = []

unique_names = ['ar', 'de', 'en', 'es', 'fi', 'fr', 'hi', 'it', 'sv', 'zh']
unique_names = ['en']

for name in unique_names:
    temp_data = []
    with open(f'mushroom.{name}-val.v1.jsonl', 'r', encoding='utf-8') as infile:
        for line in infile:
            try:
                data = json.loads(line)
                temp_data.append(data)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line}")

    # Split the temporary data into train and test sets
    #temp_train, temp_test = train_test_split(temp_data, test_size=0.2, random_state=42)

    # Append the split data to the main lists
    train_data_all.extend(temp_data)
    test_data_all.extend(temp_data)

# Create data frames
train_df = pd.DataFrame(train_data_all)
test_df = pd.DataFrame(test_data_all)
#train_df = train_df[:1]
#test_df = test_df[:1]

print("Data has been split into training and testing sets and data frames have been created.")
"""

'\n# Initialize lists for train and test data\ntrain_data_all = []\ntest_data_all = []\n\nunique_names = [\'ar\', \'de\', \'en\', \'es\', \'fi\', \'fr\', \'hi\', \'it\', \'sv\', \'zh\']\nunique_names = [\'en\']\n\nfor name in unique_names:\n    temp_data = []\n    with open(f\'mushroom.{name}-val.v1.jsonl\', \'r\', encoding=\'utf-8\') as infile:\n        for line in infile:\n            try:\n                data = json.loads(line)\n                temp_data.append(data)\n            except json.JSONDecodeError:\n                print(f"Skipping invalid JSON line: {line}")\n\n    # Split the temporary data into train and test sets\n    #temp_train, temp_test = train_test_split(temp_data, test_size=0.2, random_state=42)\n\n    # Append the split data to the main lists\n    train_data_all.extend(temp_data)\n    test_data_all.extend(temp_data)\n\n# Create data frames\ntrain_df = pd.DataFrame(train_data_all)\ntest_df = pd.DataFrame(test_data_all)\n#train_df = train_df[:1]\n#test_df = test_

In [5]:
import json

def load_jsonl_file_to_records(filename):
    records = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                records.append(json.loads(line))

        return pd.DataFrame(records)
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None


In [6]:
import pandas as pd
train_df = load_jsonl_file_to_records('mushroom.en-train_nolabel.v1.jsonl' )


test_df = load_jsonl_file_to_records('mushroom.en-val.v1.jsonl' )

In [7]:
test_df.head()

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels
0,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]"
1,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]"
2,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6, 'end': 3}, {'start'...","[[0, 3], [9, 18], [63, 70], [78, 87]]"
3,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]"
4,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]"


In [8]:
#Import the label from the sample_lable.txt
import ast

# Initialize an empty list to store all parsed label lines
hard_labels = []

# Open the file and read it line by line
with open("sample_label.txt", "r") as file:
    for line in file:
        line = line.strip()

        # Use ast.literal_eval to safely parse the string as a Python list
        parsed_line = ast.literal_eval(line)

        hard_labels.append(parsed_line)

print(hard_labels)


[[[0, 3], [22, 73]], [[0, 3], [22, 31]], [[0, 3], [22, 73]], [[0, 3], [22, 73]], [[0, 3], [22, 73]], [[49, 70]], [[49, 91]], [[30, 39], [57, 69]], [[30, 39], [57, 69]], [[34, 43], [71, 85], [95, 101], [126, 136], [144, 162]], [[11, 19], [62, 78], [83, 87]], [[11, 19], [65, 81], [88, 99], [116, 120]], [[11, 19], [65, 81], [95, 99]], [[11, 19], [62, 78], [92, 96]], [[65, 81]], [[11, 19], [65, 81], [88, 96], [100, 114]], [[11, 19], [65, 81], [85, 93], [97, 111]], [[11, 19], [65, 81], [85, 93], [97, 111]], [[11, 19], [62, 78], [83, 87]], [[11, 19], [62, 78], [85, 96], [100, 117]], [[1, 13], [49, 67]], [[48, 65]], [[47, 65]], [[4, 21], [45, 60], [64, 79], [124, 137], [141, 145]], [[48, 66]], [[1, 13], [48, 65], [69, 73]], [[1, 13], [49, 66]], [[1, 12], [48, 65], [105, 114], [118, 122]], [[1, 9], [45, 62]], [[5, 14], [124, 128]], [[5, 14], [93, 101], [129, 133]], [[5, 14], [93, 101]], [[5, 14], [82, 86]], [], [[31, 35], [46, 50]], [[31, 35], [46, 54]], [[50, 57]], [[31, 41], [52, 62]], [[49,

In [9]:
len(hard_labels)

146

In [10]:

train_df_new = train_df[:146].copy()

# Add the hard_labels to train_df_new
train_df_new["hard_labels"] = hard_labels

# Reset the index for both DataFrames to ensure they are aligned
train_df_new = train_df_new.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Filter test_df to only include rows where the "model_input" matches that of train_df_new
filtered_test_df = test_df[test_df["model_input"].isin(train_df_new["model_input"])]

print(len(train_df_new))
print(len(filtered_test_df))


146
26


In [11]:
train_df_new.head()

,lang,model_id,model_input,model_output_text,model_output_logits,model_output_tokens,hard_labels
0,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...","[[0, 3], [22, 73]]"
1,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...","[[0, 3], [22, 31]]"
2,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...","[[0, 3], [22, 73]]"
3,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...","[[0, 3], [22, 73]]"
4,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i...","[[0, 3], [22, 73]]"


In [12]:
test_df.head()

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels
0,val-en-1,EN,What did Petra van Staveren win a gold medal for?,Petra van Stoveren won a silver medal in the 2...,tiiuae/falcon-7b-instruct,"[{'start': 10, 'prob': 0.2, 'end': 12}, {'star...","[[25, 31], [45, 49], [69, 83]]"
1,val-en-2,EN,How many genera does the Erysiphales order con...,The Elysiphale order contains 5 genera.,tiiuae/falcon-7b-instruct,"[{'start': 4, 'prob': 0.2, 'end': 14}, {'start...","[[30, 31]]"
2,val-en-3,EN,Do all arthropods have antennae?,"Yes, all arachnids have antennas. However, not...",tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.6, 'end': 3}, {'start'...","[[0, 3], [9, 18], [63, 70], [78, 87]]"
3,val-en-4,EN,When did Chance the Rapper debut?,Chance the rapper debuted in 2011.,tiiuae/falcon-7b-instruct,"[{'start': 18, 'prob': 0.0909090909, 'end': 29...","[[29, 33]]"
4,val-en-5,EN,What is the UN Sustainable Development Goal 11...,The UN's Sustainable City initiative defines a...,tiiuae/falcon-7b-instruct,"[{'start': 0, 'prob': 0.0909090909, 'end': 9},...","[[70, 227]]"


In [13]:
from transformers import BertForTokenClassification, AutoTokenizer, AutoModelForTokenClassification

# Tokenizer & Model Setup
model_checkpoint = "roberta-base"
tokenizer =  AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)


# Convert Pandas DataFrame to Huggingface Dataset
train_dataset = Dataset.from_pandas(train_df_new)
test_dataset = Dataset.from_pandas(filtered_test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def tokenize_and_align_labels(example):
    """
    Tokenize and align word-level labels with subword tokens for a combined input-output scenario.
    Parameters:
    example (dict): A dictionary containing:
                     - "model_input": input question.
                     - "model_output_text": generated response.
                     - "hard_labels": list of (start, end) for words in the output.
    Returns:
    tokenized_inputs (dict): Tokenized input with aligned labels.
    """
    concatenated_text = example['model_input'] + example['model_output_text']
    max_length = 256

    tokenized_inputs = tokenizer(
        concatenated_text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_offsets_mapping=True
    )

    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'])
    offsets = tokenized_inputs['offset_mapping']

    aligned_labels = [-100] * len(tokenized_inputs['input_ids'])
    input_length =len(example['model_input'])
    output_start_idx = len(tokenizer(example['model_input'], truncation=True)['input_ids'])
    #print(output_start_idx)

    for i, (start_char, end_char) in enumerate(offsets):
        # Ignore special tokens and input tokens
        if i < output_start_idx -1 or start_char == end_char:
            continue

        label = 0
        for start, end in example['hard_labels']:
            if start_char >= (start + input_length) and end_char <= (end + input_length):
                label = 1
                break
        if i < len(aligned_labels):
            aligned_labels[i] = label

    tokenized_inputs['labels'] = aligned_labels
    tokenized_inputs['input_length'] = input_length
    tokenized_inputs['output_length'] = len(example['model_output_text'])
    tokenized_inputs['offsets'] = offsets



    return tokenized_inputs



In [15]:
# Tokenize datasets and align labels
train_dataset = train_dataset.map(tokenize_and_align_labels, batched=False)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [16]:
import numpy as np
import torch.nn.functional as F
from scipy.stats import spearmanr
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def recompute_hard_labels(soft_labels):
    """Optionally, infer hard labels from the soft labels provided"""
    hard_labels = []
    prev_end = -1
    for start, end in (
        (lbl['start'], lbl['end'])
        for lbl in sorted(soft_labels, key=lambda span: (span['start'], span['end']))
        if lbl['prob'] > 0.5
    ):
        if start == prev_end:
            hard_labels[-1][-1] = end
        else:
            hard_labels.append([start, end])
        prev_end = end
    return hard_labels

def score_iou(ref_dict, pred_dict):
    """Computes intersection-over-union between reference and predicted hard labels for a single datapoint."""
    assert ref_dict['id'] == pred_dict['id']
    ref_indices = {idx for span in ref_dict['hard_labels'] for idx in range(*span)}
    pred_indices = {idx for span in pred_dict['hard_labels'] for idx in range(*span)}
    if not pred_indices and not ref_indices: return 1.
    return len(ref_indices & pred_indices) / len(ref_indices | pred_indices)

def score_cor(ref_dict, pred_dict):
    """Computes Spearman correlation between predicted and reference soft labels for a single datapoint."""
    assert ref_dict['id'] == pred_dict['id']
    ref_vec = [0.] * ref_dict['text_len']
    pred_vec = [0.] * ref_dict['text_len']
    for span in ref_dict['soft_labels']:
        for idx in range(span['start'], min(span['end'], ref_dict['text_len'])):
            ref_vec[idx] = span['prob']
    for span in pred_dict['soft_labels']:
        for idx in range(span['start'], min(span['end'], ref_dict['text_len'])):
            pred_vec[idx] = span['prob']
    if len({round(flt, 8) for flt in pred_vec}) == 1 or len({round(flt, 8) for flt in ref_vec}) == 1:
        return float(len({round(flt, 8) for flt in ref_vec}) == len({round(flt, 8) for flt in pred_vec}))
    return spearmanr(ref_vec, pred_vec).correlation

def compute_metrics(p, dataset):
    preds = p.predictions.argmax(-1)  # Get predicted labels (indices)
    logits = p.predictions  # The raw logits for soft label evaluation
    labels = p.label_ids

    # Assuming dataset has input_length, offset_mapping, and output_length
    input_length = dataset['input_length']
    offsets = dataset['offset_mapping']
    output_length = dataset['output_length']

    # Calculate probabilities using softmax
    probabilities = F.softmax(torch.tensor(logits), dim=-1).numpy()
    probabilities = probabilities [:, :, 1]

    # Flatten the predictions and labels (ignore the -100 label used for padding/special tokens)
    prediction_labels = [pred for pred, label in zip(preds.flatten(), labels.flatten()) if label != -100]
    true_labels = [label for label in labels.flatten() if label != -100]
    '''
    # Debugging prints
    print("Logits shape:", logits.shape)
    print("Predictions shape:", preds.shape)
    print("Labels shape:", labels.shape)
    print("Offsets:", offsets)
    print("Input lengths:", input_length)
    print("Output lengths:", output_length)
    '''

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, prediction_labels, average='binary')
    acc = accuracy_score(true_labels, prediction_labels)

    # Prepare reference and predicted soft labels for custom evaluation (IoU/Spearman)
    soft_labels_ref = []
    soft_labels_pred = []

    for i in range(len(preds)):
        ref_dict = {
            'id': i,
            'soft_labels': dataset['soft_labels'][i],
            'hard_labels': dataset['hard_labels'][i],
            'text_len': output_length[i]

        }
        pred_dict = {
            'id': i,
            'soft_labels': [{'start': (s - input_length[i]), 'end': (e - input_length[i]), 'prob': pred_prob}
                            for (pred_prob, (s, e), label) in zip(probabilities[i], offsets[i], labels[i]) if label != -100],
            'hard_labels': recompute_hard_labels([{'start': (s - input_length[i]), 'end': (e - input_length[i]), 'prob': pred_prob}
                                                  for (pred_prob, (s, e), label) in zip(probabilities[i], offsets[i], labels[i]) if label != -100]),
            'text_len': output_length[i]
        }
        soft_labels_ref.append(ref_dict)
        soft_labels_pred.append(pred_dict)
        '''
        # More debugging prints
        print(f"\nExample {i}:")
        print("Predicted soft labels:", pred_dict['soft_labels'])
        print("Reference soft labels:", ref_dict['soft_labels'])
        '''

    # Calculate IoU and Spearman correlation scores
    ious = np.array([score_iou(r, d) for r, d in zip(soft_labels_ref, soft_labels_pred)])
    cors = np.array([score_cor(r, d) for r, d in zip(soft_labels_ref, soft_labels_pred)])

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'iou_mean': f'{ious.mean():.8f} ± {ious.std():.8f}',
        'spearman_mean': f'{cors.mean():.8f} ± {cors.std():.8f}',
    }



In [21]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define Training Arguments with Early Stopping
training_args = TrainingArguments(
    output_dir="./results",           # Directory for model output
    save_strategy="epoch",            # Save model at the end of each epoch
    eval_strategy="epoch",             # Evaluate model at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=28,
    per_device_eval_batch_size=8,
    num_train_epochs=10,              # Number of training epochs
    weight_decay=0.01,                # Regularization
    logging_dir='./logs',             # Directory for logging
    logging_steps=10,
    load_best_model_at_end=True,      # Load the best model at the end
    metric_for_best_model='f1',       # Metric to compare models
    greater_is_better=True,           # Specify if the metric is better when greater
)

# Initialize the Trainer object with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=lambda p: compute_metrics(p, test_dataset),  # Pass dataset to metrics
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Iou Mean,Spearman Mean
1,No log,0.456555,0.838021,0.579345,0.672515,0.508850,0.50232715 ± 0.39294334,0.46375011 ± 0.26156789
2,0.064200,0.522489,0.782735,0.564202,0.503472,0.641593,0.54755480 ± 0.37717404,0.49164787 ± 0.26795140
3,0.064200,0.540057,0.818623,0.562061,0.597015,0.530973,0.48007576 ± 0.38961754,0.46261417 ± 0.26841379
4,0.042700,0.555945,0.797284,0.584493,0.530686,0.650442,0.52037106 ± 0.38446661,0.47768279 ± 0.25402696
5,0.044200,0.517405,0.816683,0.595289,0.576763,0.615044,0.51814633 ± 0.38848682,0.47214817 ± 0.26441075
6,0.044200,0.505071,0.834142,0.603248,0.634146,0.575221,0.51117662 ± 0.39566351,0.48290379 ± 0.24904335
7,0.026400,0.523389,0.808923,0.595483,0.555556,0.641593,0.55854792 ± 0.38012751,0.48580730 ± 0.24882369
8,0.026400,0.535518,0.809893,0.593361,0.558594,0.632743,0.56183843 ± 0.38159610,0.48607792 ± 0.24773156
9,0.028700,0.544917,0.816683,0.588235,0.579399,0.597345,0.51804597 ± 0.38188715,0.47575767 ± 0.25658347


Trainer is attempting to log a value of "0.50232715 ± 0.39294334" of type <class 'str'> for key "eval/iou_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "0.46375011 ± 0.26156789" of type <class 'str'> for key "eval/spearman_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "0.54755480 ± 0.37717404" of type <class 'str'> for key "eval/iou_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "0.49164787 ± 0.26795140" of type <class 'str'> for key "eval/spearman_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "0.48007576 ± 0.38961754" of type <class 'str'> for key "eval/iou_

Trainer is attempting to log a value of "0.51117662 ± 0.39566351" of type <class 'str'> for key "eval/iou_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "0.48290379 ± 0.24904335" of type <class 'str'> for key "eval/spearman_mean" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.5050706267356873, 'eval_accuracy': 0.8341416100872939, 'eval_f1': 0.6032482598607889, 'eval_precision': 0.6341463414634146, 'eval_recall': 0.5752212389380531, 'eval_iou_mean': '0.51117662 ± 0.39566351', 'eval_spearman_mean': '0.48290379 ± 0.24904335', 'eval_runtime': 0.5973, 'eval_samples_per_second': 43.529, 'eval_steps_per_second': 6.697, 'epoch': 9.0}


In [18]:
# Define Training Arguments with Early Stopping
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    eval_strategy="epoch",  # Evaluate model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,  # Log training loss every 10 steps
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    report_to="none",  # Use none or "tensorboard" if you need logging
)

# Ensure correct loss function is passed to the Trainer.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss


TypeError: compute_metrics() missing 1 required positional argument: 'dataset'

In [ ]:
data = [1, 2, 3, 4, 5, 6]